# Basic Simulation Tutorial - Interactive Notebook

Welcome to the **WiTwin** basic simulation tutorial. This interactive notebook will guide you through creating your first wireless digital twin simulation.

## Prerequisites

Before starting, ensure you have:
- Python 3.8 or higher installed
- WiTwin package installed (`pip install witwin`)
- Basic understanding of wireless communication concepts

## Setting Up Your Environment

First, let's import the necessary modules:

In [ ]:
import witwin as wt
import numpy as np
import matplotlib.pyplot as plt

# Set up the simulation environment
env = wt.Environment(
    frequency=60e9,  # 60 GHz mmWave
    bandwidth=2e9,   # 2 GHz bandwidth
    resolution=(1024, 1024)
)

print("Environment initialized successfully!")
print(f"Frequency: {env.frequency/1e9:.1f} GHz")
print(f"Bandwidth: {env.bandwidth/1e9:.1f} GHz")
print(f"Resolution: {env.resolution}")

## Creating a Basic Scene

Now, let's create a simple scene with a transmitter and receiver:

In [ ]:
# Create transmitter
tx = wt.Transmitter(
    position=[0, 0, 2],
    orientation=[0, 0, 0],
    antenna_pattern='isotropic',
    power_dbm=30
)

# Create receiver
rx = wt.Receiver(
    position=[10, 0, 2],
    orientation=[180, 0, 0],
    antenna_pattern='directional',
    gain_dbi=20
)

# Add devices to environment
env.add_device(tx)
env.add_device(rx)

print(f"Transmitter added at position: {tx.position}")
print(f"Receiver added at position: {rx.position}")
print(f"Distance: {np.linalg.norm(np.array(tx.position) - np.array(rx.position)):.1f} m")

## Adding Environmental Objects

Let's add some objects to create multipath effects:

In [ ]:
# Add a metal reflector
reflector = wt.Object(
    shape='box',
    dimensions=[5, 0.1, 3],
    position=[5, 2, 1.5],
    material='metal',
    permittivity=1e6
)

# Add a dielectric wall
wall = wt.Object(
    shape='box',
    dimensions=[0.3, 10, 3],
    position=[7, 0, 1.5],
    material='concrete',
    permittivity=6.5
)

env.add_object(reflector)
env.add_object(wall)

print(f"Added reflector: {reflector.material} at {reflector.position}")
print(f"Added wall: {wall.material} at {wall.position}")
print(f"Total objects in environment: {len(env.objects)}")

## Running the Simulation

Now we can run the ray tracing simulation:

In [ ]:
# Configure simulation parameters
sim_config = wt.SimulationConfig(
    max_reflections=3,
    max_diffractions=1,
    ray_spacing=0.1,
    enable_doppler=True
)

print("Starting simulation...")
print(f"Max reflections: {sim_config.max_reflections}")
print(f"Max diffractions: {sim_config.max_diffractions}")
print(f"Ray spacing: {sim_config.ray_spacing} m")

# Run simulation
results = env.simulate(config=sim_config)

print("\nSimulation completed!")
print(f"Total rays traced: {results.total_rays}")
print(f"Computation time: {results.computation_time:.2f} seconds")

## Extracting Channel Information

Let's extract and analyze the channel impulse response:

In [ ]:
# Extract channel impulse response
cir = results.get_channel_impulse_response(tx, rx)

print(f"Channel impulse response extracted:")
print(f"Number of multipath components: {len(cir.delays)}")
print(f"RMS delay spread: {cir.rms_delay_spread*1e9:.2f} ns")
print(f"Mean excess delay: {cir.mean_excess_delay*1e9:.2f} ns")
print(f"K-factor: {cir.k_factor:.2f} dB")

# Show strongest paths
strongest_indices = np.argsort(np.abs(cir.amplitudes))[-5:][::-1]
print("\nStrongest 5 paths:")
for i, idx in enumerate(strongest_indices):
    delay_ns = cir.delays[idx] * 1e9
    power_db = 20 * np.log10(np.abs(cir.amplitudes[idx]))
    print(f"  {i+1}. Delay: {delay_ns:.2f} ns, Power: {power_db:.1f} dB")

## Visualizing Results

Let's create comprehensive visualizations of the simulation results:

In [ ]:
# Create comprehensive visualization
fig, axes = plt.subplots(2, 2, figsize=(15, 12))

# Plot 1: Channel Impulse Response
axes[0, 0].stem(cir.delays * 1e9, 20 * np.log10(np.abs(cir.amplitudes)), basefmt=' ')
axes[0, 0].set_xlabel('Delay (ns)')
axes[0, 0].set_ylabel('Power (dB)')
axes[0, 0].set_title('Channel Impulse Response')
axes[0, 0].grid(True, alpha=0.3)

# Plot 2: Power Delay Profile
axes[0, 1].plot(cir.delays * 1e9, 20 * np.log10(np.abs(cir.amplitudes)), 'o-')
axes[0, 1].set_xlabel('Delay (ns)')
axes[0, 1].set_ylabel('Power (dB)')
axes[0, 1].set_title('Power Delay Profile')
axes[0, 1].grid(True, alpha=0.3)

# Plot 3: Ray paths (top view)
results.plot_rays_2d(tx, rx, ax=axes[1, 0], view='top')
axes[1, 0].set_title('Ray Paths - Top View')
axes[1, 0].set_aspect('equal')

# Plot 4: Angle of arrival histogram
aoa = results.get_angles_of_arrival(tx, rx)
axes[1, 1].hist(aoa * 180 / np.pi, bins=30, alpha=0.7)
axes[1, 1].set_xlabel('Angle of Arrival (degrees)')
axes[1, 1].set_ylabel('Number of Rays')
axes[1, 1].set_title('Angle of Arrival Distribution')
axes[1, 1].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

print("Visualization complete!")

## Performance Analysis

Let's analyze the communication performance metrics:

In [ ]:
# Calculate performance metrics
path_loss = results.get_path_loss(tx, rx)
snr = results.get_snr(tx, rx, noise_figure=5)  # 5 dB noise figure
capacity = results.get_channel_capacity(tx, rx)
coherence_bandwidth = results.get_coherence_bandwidth(tx, rx)

print("Communication Performance Metrics:")
print(f"Path Loss: {path_loss:.2f} dB")
print(f"Signal-to-Noise Ratio: {snr:.2f} dB")
print(f"Channel Capacity: {capacity/1e6:.2f} Mbps")
print(f"Coherence Bandwidth: {coherence_bandwidth/1e6:.2f} MHz")

# Compare with free space path loss
distance = np.linalg.norm(np.array(tx.position) - np.array(rx.position))
free_space_loss = 20 * np.log10(distance) + 20 * np.log10(env.frequency) + 20 * np.log10(4 * np.pi / 3e8)
excess_loss = path_loss - free_space_loss

print(f"\nComparison with Free Space:")
print(f"Free Space Path Loss: {free_space_loss:.2f} dB")
print(f"Excess Loss due to Environment: {excess_loss:.2f} dB")

## Advanced Analysis: Frequency Response

Let's examine the frequency response characteristics:

In [ ]:
# Calculate frequency response
frequencies = np.linspace(-env.bandwidth/2, env.bandwidth/2, 1024)
freq_response = results.get_frequency_response(tx, rx, frequencies)

# Plot magnitude and phase response
fig, (ax1, ax2) = plt.subplots(2, 1, figsize=(12, 8))

# Magnitude response
ax1.plot(frequencies/1e6, 20 * np.log10(np.abs(freq_response)))
ax1.set_xlabel('Frequency Offset (MHz)')
ax1.set_ylabel('Magnitude (dB)')
ax1.set_title('Channel Frequency Response - Magnitude')
ax1.grid(True, alpha=0.3)

# Phase response
ax2.plot(frequencies/1e6, np.angle(freq_response) * 180 / np.pi)
ax2.set_xlabel('Frequency Offset (MHz)')
ax2.set_ylabel('Phase (degrees)')
ax2.set_title('Channel Frequency Response - Phase')
ax2.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

# Calculate channel statistics
channel_std = np.std(20 * np.log10(np.abs(freq_response)))
coherence_bw = results.get_coherence_bandwidth(tx, rx, threshold=0.5)

print(f"Channel variability (std): {channel_std:.2f} dB")
print(f"50% coherence bandwidth: {coherence_bw/1e6:.2f} MHz")

## Summary and Next Steps

🎉 **Congratulations!** You've successfully completed the basic WiTwin simulation tutorial.

### What you've learned:
- ✅ Set up a WiTwin simulation environment
- ✅ Created wireless devices (transmitter and receiver)
- ✅ Added environmental objects with different materials
- ✅ Configured and ran ray tracing simulations
- ✅ Analyzed channel characteristics and performance metrics
- ✅ Created comprehensive visualizations

### Next Steps:
1. **Advanced Tutorials**: Try the [Radar Configuration](/docs/tutorials/radar-configuration) tutorial
2. **Dynamic Simulations**: Learn about time-varying scenarios
3. **Optimization**: Explore differentiable simulation features
4. **Custom Materials**: Define your own material properties

### Resources:
- 📚 [Full API Documentation](/docs/api)
- 🎯 [Advanced Examples](/docs/advanced)
- 💬 [Community Forum](https://forum.witwin.ai)
- 🐛 [Report Issues](https://github.com/witwin/issues)

Happy simulating! 🚀

In [ ]:
# Final cleanup and environment summary
print("Simulation Environment Summary:")
print(f"Environment ID: {env.id}")
print(f"Devices: {len(env.devices)}")
print(f"Objects: {len(env.objects)}")
print(f"Frequency: {env.frequency/1e9:.1f} GHz")
print(f"Bandwidth: {env.bandwidth/1e9:.1f} GHz")
print(f"Total simulation time: {results.computation_time:.2f} seconds")
print("\nThank you for using WiTwin! 🎉")